In [24]:
from pymongo import MongoClient
from bson.son import SON
import pprint as pp
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import date, timedelta

In [25]:
client = MongoClient("mongodb://localhost:27017/")
db = client.enron_mail
messages = db.messages

In [26]:
lowDate = dt.datetime(1998,11,1)
highDate = dt.datetime(2004,1,1)

doc = messages.find_one()
pp.pprint(doc)

pipelineTo = [{"$match":{"headers.Date":{"$gt":lowDate,"$lt":highDate}}},{"$unwind":"$headers.To"},\
			{"$group":{"_id":{"to":"$headers.To","year":{"$year":"$headers.Date"},"week":{"$week":"$headers.Date"}},"cnt":{"$sum":1}}},\
			{"$sort":{"_id.to":1}},\
			{"$sort":{"cnt":-1}},\
			{"$sort":{"_id.week":1}},\
			{"$sort":{"_id.year":1}}]
			#{"$limit":500000}]

pipelineFrom = [{"$match":{"headers.Date":{"$gt":lowDate,"$lt":highDate}}},\
			{"$group":{"_id":{"from":"$headers.From","year":{"$year":"$headers.Date"},"week":{"$week":"$headers.Date"}},"cnt":{"$sum":1}}},\
			{"$sort":{"_id.from":1}},\
			{"$sort":{"cnt":-1}},\
			{"$sort":{"_id.week":1}},\
			{"$sort":{"_id.year":1}}]
			#{"$limit":500000}]

currentPipeline = pipelineFrom
currentPipelineKey = "from"


{u'_id': ObjectId('59b35dc41a62b724314170e5'),
 u'body': u'Here is our forecast\n\n ',
 u'filename': u'1.',
 u'headers': {u'Content-Transfer-Encoding': u'7bit',
              u'Content-Type': u'text/plain; charset=us-ascii',
              u'Date': datetime.datetime(2001, 5, 14, 23, 39),
              u'From': u'phillip.allen@enron.com',
              u'Message-ID': u'<18782981.1075855378110.JavaMail.evans@thyme>',
              u'Mime-Version': u'1.0',
              u'Subject': u'',
              u'To': u'tim.belden@enron.com',
              u'X-FileName': u'pallen (Non-Privileged).pst',
              u'X-Folder': u"\\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
              u'X-From': u'Phillip K Allen',
              u'X-Origin': u'Allen-P',
              u'X-To': u'Tim Belden <Tim Belden/Enron@EnronXGate>',
              u'X-bcc': u'',
              u'X-cc': u''},
 u'mailbox': u'allen-p',
 u'subFolder': u'_sent_mail'}


In [27]:
results = messages.aggregate(currentPipeline, allowDiskUse=True)

In [29]:
runningWeek = 0
last_week_last_date = date(1998,10,1)
timeDictionary = dict()
currentRankList = list()

In [30]:
def get_start_end_dates(year, week):
    '''
    Courtesy: http://stackoverflow.com/questions/30656008/get-start-date-and-end-date-of-the-week-given-week-number-and-year
    
    Parameter:
    The Year and the week number
    
    Returns:
    A tuple with start date and end date of that week
    '''
    d = date(year,1,1)
    if(d.weekday()<= 3):
        d = d - timedelta(d.weekday())             
    else:
        d = d + timedelta(7-d.weekday())
    dlt = timedelta(days = (week-1)*7)
    return d + dlt,  d + dlt + timedelta(days=6)

In [31]:
all_date_list = list()
all_date_set = set()
rankList = list()
currentRankList = list()
for result in results:
    ## Create a 2d list, each row contains recipient mail id in descending order
    ## of their rank based on how many emails they sent or received
    weekCurrentDocument = result["_id"]["week"]
    yearCurrentDocument = result["_id"]["year"]
    current_last_date_of_week = get_start_end_dates(yearCurrentDocument, weekCurrentDocument)[1]
    ## using second index of the function because we are interested about the last date of that week
    
    if current_last_date_of_week > last_week_last_date:
        runningWeek += 1
        print "runningWeek ", runningWeek
        rankList.append(currentRankList)
        last_week_last_date = current_last_date_of_week
        currentRankList = list()
        currentRankList.append([result["_id"][currentPipelineKey], str(result["cnt"])])
        all_date_list.append(current_last_date_of_week)
    else:
        currentRankList.append([result["_id"][currentPipelineKey], str(result["cnt"])])
    #pp.pprint(result)

runningWeek  1
runningWeek  2
runningWeek  3
runningWeek  4
runningWeek  5
runningWeek  6
runningWeek  7
runningWeek  8
runningWeek  9
runningWeek  10
runningWeek  11
runningWeek  12
runningWeek  13
runningWeek  14
runningWeek  15
runningWeek  16
runningWeek  17
runningWeek  18
runningWeek  19
runningWeek  20
runningWeek  21
runningWeek  22
runningWeek  23
runningWeek  24
runningWeek  25
runningWeek  26
runningWeek  27
runningWeek  28
runningWeek  29
runningWeek  30
runningWeek  31
runningWeek  32
runningWeek  33
runningWeek  34
runningWeek  35
runningWeek  36
runningWeek  37
runningWeek  38
runningWeek  39
runningWeek  40
runningWeek  41
runningWeek  42
runningWeek  43
runningWeek  44
runningWeek  45
runningWeek  46
runningWeek  47
runningWeek  48
runningWeek  49
runningWeek  50
runningWeek  51
runningWeek  52
runningWeek  53
runningWeek  54
runningWeek  55
runningWeek  56
runningWeek  57
runningWeek  58
runningWeek  59
runningWeek  60
runningWeek  61
runningWeek  62
runningWeek  63
r

In [32]:
len(rankList)

196

In [33]:
len(all_date_list)

196

In [34]:
directory = "data_rank_enron_weekly"
if not os.path.exists(directory):
    os.makedirs(directory)

In [35]:
filename_all_dates = 'time_list_weekly_enron_mail_from.txt'
with open(filename_all_dates,'w') as f_date:
    for i,date in enumerate(all_date_list):
        current_date_string = date.strftime('%d.%m.%Y')
        f_date.write(current_date_string + "\n")
        filename_rank_current_week = directory + "/" + current_date_string
        with open(filename_rank_current_week, 'w') as f_weekly_rank:
            for mail_address_and_count in rankList[i]:
                f_weekly_rank.write("\t".join(mail_address_and_count)+"\n")